In [12]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import glob
import os
import math
import csv


In [14]:
lat_lon_file_path='/Users/moctader/Thesis_code/lat_lon/results.csv'
feature_vector_path='/Users/moctader/Thesis_code/lat_lon/feature_vector.csv'
csv_file_path = '/Users/moctader/Thesis_code/lat_lon/results.csv'
tiles_template = "/Users/moctader/Thesis/{t}/{z}/{x}/{y}.png"
combined_csv_path = '/Users/moctader/Thesis_code/lat_lon/combined_results.csv'
points = gpd.read_file("/Users/moctader/Thesis_code/GTK_ASsoil_obs.csv")
t_values = glob.glob('/Users/moctader/Thesis/*')


In [5]:
points.POINT_X = points.POINT_X.astype("float")
points.POINT_Y = points.POINT_Y.astype("float")
zoom_level = 10


In [6]:
samples = gpd.GeoDataFrame(
    points.CLASS, crs="EPSG:3067", geometry=gpd.points_from_xy(points.POINT_X, points.POINT_Y)
).to_crs("WGS84")

tile_list = [(point.x, point.y) for point in samples['geometry']]

In [8]:

results_list = []

def project(p, zoom, point_class):
    lon, lat = p.geometry.x, p.geometry.y
    results_list.append([lon, lat])

    siny = np.sin(lat * np.pi / 180)
    siny = min(max(siny, -0.9999), 0.9999)

    x = TILE_SIZE * (0.5 + lon / 360)
    y = TILE_SIZE * (0.5 - np.log((1 + siny) / (1 - siny)) / (4 * np.pi))

    scale = 2**zoom

    tx = x * scale // TILE_SIZE
    ty = y * scale // TILE_SIZE

    px = x * scale % TILE_SIZE // 1
    py = y * scale % TILE_SIZE // 1

    return (int(zoom), int(tx), int(ty)), (px, py), point_class

In [10]:


TILE_SIZE = 256

for i in range(samples.shape[0]):
    for t_value in t_values:
        if not t_value.endswith('.zip'):
            t_value = os.path.splitext(os.path.basename(t_value))[0]
           
            try:
                project(samples.iloc[i], zoom_level, samples['CLASS'][i])
                
               
            except FileNotFoundError:
                print(f"No valid {t_value} found for sample {i}  ------> {tz}/{tx}/{ty}  ")
                pass
            
            

In [13]:
columns = ['lon', 'lat']
# Convert the results_list to a DataFrame
df = pd.DataFrame(results_list, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)
results_list
print(df)

             lon        lat
0      25.768938  64.777988
1      25.768938  64.777988
2      25.768938  64.777988
3      25.768938  64.777988
4      25.768938  64.777988
...          ...        ...
58236  25.159435  65.626582
58237  25.159435  65.626582
58238  25.159435  65.626582
58239  25.159435  65.626582
58240  25.159435  65.626582

[58241 rows x 2 columns]


In [15]:
# Load the data from CSV files into DataFrames
df1 = pd.read_csv(lat_lon_file_path)
df2 = pd.read_csv(feature_vector_path)

# Concatenate the two DataFrames along columns
combined_df = pd.concat([df2, df1], axis=1)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(combined_csv_path, index=False)

# Display the combined DataFrame
print(combined_df.head(2))

          0        1    2         3         4         5         6         7  \
0  0.030598  4.98712  0.0  0.564867  1.535341  1.688056  0.365771  0.758600   
1  0.009759  4.86358  0.0  0.557743  1.560788  1.973111  0.537939  0.504111   

          8         9  ...       993  994       995       996       997  \
0  0.022976  4.397676  ...  0.006134  0.0  1.378555  0.159766  0.000000   
1  0.004016  4.126801  ...  0.058373  0.0  1.531437  0.271479  0.000056   

        998       999  label        lon        lat  
0  4.429381  9.502254    1.0  25.768938  64.777988  
1  4.403303  9.131659    1.0  25.768938  64.777988  

[2 rows x 1003 columns]
